# **I. Set up environment**

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops
!pip install -q -U googletrans==3.1.0a0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.0 MB/s eta 0:00:0

In [ ]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from googletrans import Translator
from pprint import pprint
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
notebook_login()

# **II. Load pre-trained LLM**

In [ ]:
MODEL_NAME = "vilm/vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model-00001-of-00003.safetensors:  86%|########6 | 4.28G/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
print(generation_config)

trainable params: 39,976,960 || all params: 6,895,579,136 || trainable%: 0.5797
GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 46303,
  "max_length": 4096,
  "max_memory": {
    "cpu": 329543315456
  },
  "max_new_tokens": 200,
  "no_split_module_classes": [
    "LlamaDecoderLayer"
  ],
  "pad_token_id": 46303,
  "special_dtypes": {},
  "temperature": 0.7,
  "top_p": 0.7
}



# III. **Fine-tuning LLM**

## **1. Prepare dataset**

In [ ]:
data = load_dataset('hllj/vi_grade_school_math_mcq')
data

README.md:   0%|          | 0.00/2.95k [00:00<?, ?B/s]

vietjack.json:   0%|          | 0.00/4.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2733 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['grade', 'id', 'title', 'problems', 'url'],
        num_rows: 2733
    })
})

In [ ]:
def generate_prompt(question, choices, explanation):
  return f"""
<|im_start|>system
Bạn là một chuyên gia toán học. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|>user
### Câu hỏi:
{question}
### Các lựa chọn
{choices}
### Câu trả lời:

<|im_start|>assistant
{explanation}
  """.strip()

def generate_and_tokenize_prompt(question, choices, explanation):
  full_prompt = generate_prompt(question, choices, explanation)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)

  return tokenized_full_prompt

In [ ]:
from tqdm import tqdm

training_samples = []
for sample in tqdm(data['train']):
  for quest in sample['problems']:
    choices = quest['choices']
    explanation = quest['explanation'].strip()
    question = quest['question']

    if explanation == '' or question == '' or choices == []:
      continue

    try:
      question = question.split('\n \n')[1].strip()
    except:
      continue

    choices = '\n'.join(choices)
    training_sample = generate_and_tokenize_prompt(question, choices, explanation)
    training_samples.append(training_sample)

training_samples[:2]

100%|██████████| 2733/2733 [00:04<00:00, 591.64it/s]


[{'input_ids': [1, 46304, 1788, 13, 39834, 18916, 32039, 32633, 32177, 32858, 32212, 29889, 33755, 32145, 32200, 32889, 32723, 35410, 32863, 33471, 32161, 32017, 33367, 32742, 29892, 33638, 32271, 4331, 491, 4331, 32646, 28810, 32006, 32742, 32379, 32132, 32669, 29889, 13, 13, 46304, 1404, 13, 2277, 29937, 34445, 32723, 29901, 13, 30250, 30902, 32654, 32009, 32551, 34795, 41788, 18916, 29901, 13, 2277, 29937, 32541, 33367, 32742, 13, 29909, 29889, 32410, 7477, 13, 29933, 29889, 32433, 7477, 13, 29907, 29889, 32476, 7477, 13, 29928, 29889, 32388, 7477, 13, 2277, 29937, 34445, 32550, 32529, 29901, 13, 13, 46304, 408, 46043, 29873, 424, 13, 45583, 32523, 32271, 29871, 13, 37235, 32132, 32669, 18916, 29901, 315, 29871, 13, 34079, 32366, 32201, 34313, 11062, 32343, 32074, 32654, 32009, 32551, 34795, 41788, 18916, 32476, 7477, 869], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
choices_data = Dataset.from_list(training_samples)
choices_data

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 9173
})

## **2. Training**

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      max_steps=1000,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=choices_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:501: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch

Step,Training Loss
1,3.303800
2,3.890600
3,4.029200
4,3.781200
5,3.864100
6,3.776900
7,3.609600
8,3.306800
9,3.340300
10,2.977400


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


TrainOutput(global_step=1000, training_loss=0.6635894245505333, metrics={'train_runtime': 6470.7983, 'train_samples_per_second': 0.618, 'train_steps_per_second': 0.155, 'total_flos': 2.1468688149774336e+16, 'train_loss': 0.6635894245505333, 'epoch': 0.43606235691703915})

## **3. Prediction**

In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = """
<|im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|>user
### Câu hỏi:
Số gồm 1 đơn vị và 2 chục đọc là :
### Các lựa chọn:
A. 20
B. 21
C. 30
D. 31
### Câu trả lời:

<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|im_start|> system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|> user
### Câu hỏi:
Số gồm 1 đơn vị và 2 chục đọc là :
### Các lựa chọn:
A. 20
B. 21
C. 30
D. 31
### Câu trả lời:

<|im_start|> assistant
Đáp án B. 21 Đáp án cần chọn là: B. 21 Số gồm 1 đơn vị và 2 chục đọc là: hai mươi mốt. Đáp án cần chọn là: B. 21 . Vậy đáp án đúng là: B. 21 . Chọn B Lưu ý : Số gồm 1 đơn vị và 2 chục đọc là: hai mươi mốt. Chọn B. 21 Lưu ý : Số gồm 1 đơn vị và 2 chục đọc là: hai mươi mốt. Chọn B. 21 Lưu ý : Số gồm 1 đơn vị và 2 chục đọc là: hai mươi mốt. Chọn B. 21 Lưu ý : Số gồm 1 đơn vị và 2 chục đọc là: hai mươi mốt. Chọn B. 21 Lưu ý : Số gồm 1 đơn vị và 2 chục đọc là: hai mươi mốt. Chọn B. 21 Lưu ý : Số gồm 1 đơn vị và 2 chục đọc là: hai mươi mốt. Chọn B. 21 Lưu ý : Số gồm 1 đơn vị và 2 chục đọc là: hai
CPU times: user 2min 42s, sys: 1min 27s, total: 4min 10s
Wall time: 4min 12s


## **4. Save model**

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "NhatNguyen2101/vinallama-peft-7b-math-solver"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NhatNguyen2101/vinallama-peft-7b-math-solver/commit/e6044be2946339b29a80981cbdc575ca9c5efe7b', commit_message='Upload model', commit_description='', oid='e6044be2946339b29a80981cbdc575ca9c5efe7b', pr_url=None, pr_revision=None, pr_num=None)

# **IV. Inference**

In [ ]:
PEFT_MODEL = "NhatNguyen2101/vinallama-peft-7b-math-solver"

config = PeftConfig.from_pretrained(PEFT_MODEL)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

adapter_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = model.config.pad_token_id
generation_config.eos_token_id = model.config.eos_token_id

In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = """
<|im_start|>system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|>user
### Câu hỏi:
Số gồm 1 đơn vị và 3 chục đọc là :
### Các lựa chọn:
A. 30
B. 31
C. 20
D. 21
### Câu trả lời:

<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<|im_start|> system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

<|im_start|> user
### Câu hỏi:
Số gồm 1 đơn vị và 3 chục đọc là :
### Các lựa chọn:
A. 30
B. 31
C. 20
D. 21
### Câu trả lời:

<|im_start|> assistant
Đáp án B. 31 Số gồm 1 đơn vị và 3 chục đọc là: ba mươi. Đáp án cần chọn là: B. 31. Chọn B Lưu ý : Số gồm 1 đơn vị và 3 chục đọc là: ba mươi. Đáp án cần chọn là: B. 31. Chọn B. 31. Lưu ý : Số gồm 1 đơn vị và 3 chục đọc là: ba mươi. Đáp án cần chọn là: B. 31. Chọn B. 31. Lưu ý : Số gồm 1 đơn vị và 3 chục đọc là: ba mươi. Đáp án cần chọn là: B. 31. Chọn B. 31. Lưu ý : Số gồm 1 đơn vị và 3 chục đọc là: ba mươi. Đáp án cần chọn là: B. 31. Chọn B. 31. Lưu ý : Số gồm 1 đơn vị và 3 chục đọc là: ba mươi. Đáp án cần chọn là: B. 31. Chọn B. 31. Lưu ý : Số gồm 1 đơn
CPU times: user 30.3 s, sys: 465 ms, total: 30.8 s
Wall time: 31.1 s
